In [1]:
import pylab as plt
import numpy as np
import pandas as pd
import matplotlib as mpl
#import proposal as pp  #installed with pip
import scipy
from scipy import interpolate
from scipy.interpolate import InterpolatedUnivariateSpline
from scipy.interpolate import interp2d
import scipy.integrate as integrate
# matplotlib used plotting. Not required to run the code.
import matplotlib.pyplot as plt
import re
import sys
import glob
#from MCEq.core import config, MCEqRun
#import crflux.models as crf
import random
try:
    import cPickle as pickle
except ImportError:
    import pickle

In [3]:
r=0

#pull in all of the MCeQoutputs to put into one dataframe
for files in glob.glob('/Users/rogerslc/Documents/MuFluxPred/NewInteractionModel/MCeQouput*.h5'):

    if r==0:
        mussimData=pd.read_hdf(files)

        r+=1
    else:
        df=pd.read_hdf(files)

        mussimData=mussimData.append(df, ignore_index=True)
        
mussimData=mussimData.rename(columns={"theta": "alpha", "phis":  "beta"})
        
#mussimData=pd.read_hdf('/Users/rogerslc/Documents/MuFluxPred/MCeQoutputJustValidRegion.h5','Muons')

mussimData.beta=np.deg2rad(mussimData.beta)
mussimData.alpha=np.deg2rad(mussimData.alpha)

In [4]:
#need a list of alphas and betas to interpolate between
listofpoints_alpha=[]
listofpoints_beta=[]
tmpdict={}
alphas = mussimData.alpha
betas = mussimData.beta

for i in range(0,len(alphas)):
     tmpdict[alphas[i],betas[i]]=True

AllAlphasBetas=tmpdict.keys()
for k in AllAlphasBetas:
    listofpoints_alpha.append(k[0])
    listofpoints_beta.append(k[1])
    
    
UnitVectors=[]
d2r=1

# Make 3D unit vectors on the sphere for each point available
for i in range(0, len(listofpoints_alpha)):
    UnitVectors.append([np.sin(listofpoints_alpha[i]*d2r)*np.cos(listofpoints_beta[i]*d2r),np.sin(listofpoints_alpha[i]*d2r)*np.sin(listofpoints_beta[i]*d2r),np.cos(listofpoints_alpha[i]*d2r)]) 

In [5]:
#This function finds the closest available (a,b) on the sphere to the one we have provided
def ClosestPoint(a,b):
    ThisUnitVector=[np.sin(a)*np.cos(b),np.sin(a)*np.sin(b),np.cos(a)]
    dotprods=[np.abs(ThisUnitVector[0]*UnitVectors[i][0]+ThisUnitVector[1]*UnitVectors[i][1]+ThisUnitVector[2]*UnitVectors[i][2]) for i in range(0,len(UnitVectors))]
    closest=np.argmax(dotprods)
    return listofpoints_alpha[closest],listofpoints_beta[closest],closest

In [6]:
#Get a continously interpolated F(E) function for nearest a and b
def GetEnergyFunction(a,b,EPower=3):
    ECut=1e5
    besta,bestb,idx=ClosestPoint(a,b)
    dataSubSet=mussimData[(mussimData.alpha==besta) & (mussimData.beta==bestb) & (mussimData.Energies<ECut)]
    interpfun= interpolate.interp1d(dataSubSet.Energies,dataSubSet.MuFLux)
    returnfunction=lambda E: interpfun(E)/E**EPower
    return returnfunction

In [7]:
#Integrate the continuous flux function near to (a,b) between these limits
def GetFluxIntegral(a,b,EMin,EMax):
    EnergyFunction=GetEnergyFunction(a,b)
    return integrate.quad(EnergyFunction,EMin,EMax)

In [8]:
#energies to get fluxes over. Change this to match the energies Proposal had been run over
BinCenters=np.array([100,300,400,450,500,550,600,650,700,800,1000,2000,4000,6000])


EBins=(BinCenters[1:]+BinCenters[:-1])/2

In [9]:
#Get muon fluxes vs energy mesh
numofstps=50
cosa_sample=np.linspace(0,1,numofstps)
a_sample=np.arccos(cosa_sample)
b_sample=np.linspace(-np.pi,np.pi,numofstps)
cosa_mesh, b_mesh=np.meshgrid(cosa_sample,b_sample)
fluxmeshes=[]


#for iE in range(0,len(EBins)-1):
for iE in range(0,2):

    fluxmesh = np.zeros_like(cosa_mesh)
    print("Integrating fluxes for "+ str(EBins[iE])+ " < E < " + str(EBins[iE+1]))
    for ia in range(0,len(a_sample)):
        for ib in range(0,len(b_sample)):
            a=a_sample[ia]
            b=b_sample[ib]
            fluxmesh[ia,ib] = GetFluxIntegral(a,b,EBins[iE],EBins[iE+1])[0]
    fluxmeshes.append(fluxmesh)

Integrating fluxes for 200.0 < E < 350.0
Integrating fluxes for 350.0 < E < 425.0


In [10]:
#save the output
f=open("./MCEqFluxMeshes.pkl","wb")
pickle.dump([BinCenters,cosa_sample,b_sample,fluxmeshes_fine],f)
f.close()

'f=open("./MCEqFluxMeshes.pkl","wb")\npickle.dump([BinCenters,cosa_sample,b_sample,fluxmeshes_fine],f)\nf.close()'

In [18]:
#these are for creating the interpolation function between all alpha and energy

k=mussimData.alpha.unique()
k.sort()

df=mussimData[(mussimData.MuFLux>10**-9)]
Es=df.Energies.unique()
Es.sort()
    
alt=1431   
flxs=[] 
for alpha in k:

    for nrg in Es:
        flxs.append(mussimData[(mussimData.alpha==alpha)&(mussimData.Energies==nrg)].MuFLux.iloc[0])           

        
flxarray=np.array(flxs)
flxarray = flxarray.reshape(len(k), len(Es))
FluxMuons=scipy.interpolate.RectBivariateSpline(k,Es,flxarray,s=0,kx=3, ky=3)



"with open('./MCeQ_Muons_interpolator_ALL.pkl', 'wb') as f:\n    pickle.dump(FluxMuons, f)"

In [ ]:
#save interpolation function
with open('./MCeQ_Muons_interpolator_ALL.pkl', 'wb') as f:
    pickle.dump(FluxMuons, f)